In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from datetime import datetime, time
import pandas_profiling

In [2]:
#!pip install pandas_profiling

In [3]:
#sns.set(rc = {'figure.figsize':(35,8)})

In [4]:
df = pd.read_excel('Flight_data.xlsx')

In [5]:
df.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [7]:
df.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [7]:
df['Route'].isnull().any()

True

In [8]:
df['Total_Stops'].isnull().any()

True

### Univariate Analysis

In [9]:
df['Airline'].value_counts()

Jet Airways                          3849
IndiGo                               2053
Air India                            1752
Multiple carriers                    1196
SpiceJet                              818
Vistara                               479
Air Asia                              319
GoAir                                 194
Multiple carriers Premium economy      13
Jet Airways Business                    6
Vistara Premium economy                 3
Trujet                                  1
Name: Airline, dtype: int64

In [10]:
ax = sns.countplot(x= 'Airline', data = df)
plt.xticks(rotation = 90)
for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

In [11]:
df['Source'].value_counts()

Delhi       4537
Kolkata     2871
Banglore    2197
Mumbai       697
Chennai      381
Name: Source, dtype: int64

In [12]:
ax = sns.countplot(df['Source'])
ax.set_xticklabels(ax.get_xticklabels(),rotation = 45)

for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

C:\Users\Sudheesh E\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [13]:
df['Destination'].value_counts()

Cochin       4537
Banglore     2871
Delhi        1265
New Delhi     932
Hyderabad     697
Kolkata       381
Name: Destination, dtype: int64

In [14]:
df['Route'].value_counts().nlargest(30)

DEL → BOM → COK          2376
BLR → DEL                1552
CCU → BOM → BLR           979
CCU → BLR                 724
BOM → HYD                 621
CCU → DEL → BLR           565
BLR → BOM → DEL           402
MAA → CCU                 381
DEL → HYD → COK           326
DEL → JAI → BOM → COK     240
DEL → BLR → COK           232
DEL → COK                 213
DEL → AMD → BOM → COK     205
DEL → MAA → COK           146
DEL → IDR → BOM → COK      99
DEL → HYD → MAA → COK      83
DEL → NAG → BOM → COK      71
CCU → MAA → BLR            65
DEL → GOI → BOM → COK      58
DEL → HYD → BOM → COK      56
CCU → HYD → BLR            53
CCU → BBI → BLR            50
DEL → BHO → BOM → COK      50
CCU → IXR → DEL → BLR      46
DEL → CCU → BOM → COK      44
DEL → PNQ → COK            42
CCU → BBI → BOM → BLR      42
CCU → GAU → BLR            41
DEL → LKO → BOM → COK      41
DEL → ATQ → BOM → COK      40
Name: Route, dtype: int64

In [15]:
df['Total_Stops'].value_counts()

1 stop      5625
non-stop    3491
2 stops     1520
3 stops       45
4 stops        1
Name: Total_Stops, dtype: int64

In [16]:
df['Additional_Info'].value_counts()

No info                         8345
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
No Info                            3
Red-eye flight                     1
1 Short layover                    1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [17]:
sns.histplot(df['Price'], kde= True, color= 'red')

<AxesSubplot:xlabel='Source', ylabel='count'>

### Bivariate Analysis

In [18]:
sns.set(rc = {'figure.figsize':(15,5)})

df.Date_of_Journey = pd.to_datetime(df['Date_of_Journey'], format = '%d/%m/%Y')
sns.lineplot(x = df['Date_of_Journey'], y = df['Price'])

<AxesSubplot:xlabel='Source', ylabel='count'>

In [19]:
sns.relplot(y='Airline', x='Price', data=df, height=3, aspect=6/3)

In [20]:
sns.relplot(y='Date_of_Journey', x='Price', data=df, height=8, aspect=3/1 )

In [21]:
sns.relplot(y='Source', x='Price', data=df, height=3, aspect=6/3 )

In [22]:
sns.relplot(y='Destination', x='Price', data=df, height=3, aspect=6/3 )

In [23]:
sns.relplot(y='Total_Stops', x='Price', data=df, height=3, aspect=6/3 )

In [24]:
sns.relplot(y='Additional_Info', x='Price', data=df, height=3, aspect=8/3)

### MultiVariate Analysis

In [25]:
sns.relplot(x='Airline', y='Price', data=df, height=8, aspect=14/5, hue='Additional_Info', palette='rocket')

In [26]:
sns.relplot(x='Airline', y='Price', data=df, height=5, aspect=14/4, hue='Total_Stops', palette='rocket')

In [27]:
sns.relplot(x='Airline', y='Price', data=df, height=8, aspect=14/4, hue='Source', palette='rocket')

In [28]:
sns.relplot(x='Airline', y='Price', data=df, height=8, aspect=14/4, hue='Destination', palette='rocket')

### Creating, Modifying and Droping columns

#### Converting data types few column

In [29]:
df.Date_of_Journey = pd.to_datetime(df['Date_of_Journey'], format = '%d/%m/%Y')

In [30]:
# Creating 'Month_of_Journey' , 'Day_of_Journey', 'Year_of_Journey' columns from 'Date_of_Journey'

df['Month_of_Journey'] = df['Date_of_Journey'].dt.month
df['Day_of_Journey'] = df['Date_of_Journey'].dt.day
df['Weekday_of_Journey'] = df['Date_of_Journey'].dt.weekday

In [31]:
# Dropping 'Date_of_Journey' column
df.drop('Date_of_Journey', axis= 'columns', inplace= True)

In [32]:
# Creating 'Arr_Time_Hour' and 'Arr_Time_Min' from 'Arrival_Time' column 
# We are not using this instead we use Part_of _the Day
"""
df ['x_col'] = df['Arrival_Time'].str.split(expand=True).iloc[:,0]

df['Arr_Time_Hour'] = df['x_col'].str.split(':', expand= True).iloc[:,0]
df['Arr_Time_Min'] = df['x_col'].str.split(':', expand= True).iloc[:,1]
"""
# Creating 'Dep_Time_Hour' and 'Dep_Time_Min' columns from 'Dep_Time'
# We are not using this instead we use Part_of _the Day
"""
df['x_col'] = df['Dep_Time'].str.split(expand=True)
df['Dep_Time_Hour'] = df['x_col'].str.split(':', expand= True).iloc[:,0]
df['Dep_Time_Min'] = df['x_col'].str.split(':', expand= True).iloc[:,1]

"""

## Creating a function to extract whether the journey is on 'Morning, Afternoon, Evening or Night'

def dep_parts_of_the_day(x):
    
    x = x.strip()
    t = (int)(x.split(':')[0])
    
    if (t>=5 and t<11):
        x = 'morning'
    elif(t>=11 and t<16):
        x='afternoon'
    elif(t>=16 and t<21):
        x='evening'
    elif(t>=21 or t<5):
        x='night'
        
    return x


df['Dep_Time'] = df['Dep_Time'].apply(dep_parts_of_the_day)
df['Arrival_Time']=df['Arrival_Time'].apply(dep_parts_of_the_day)

In [33]:
# Dropping 'Route' column
df.drop('Route', axis='columns', inplace=True)

In [34]:
# Dropping the row which have 'nan' in 'Total_Stops' 'Duration' columns

df.dropna(subset=['Total_Stops'],inplace= True)
df.dropna(subset=['Duration'], inplace=True)

In [35]:
# Mapping the Total_Stops column 
df['Total_Stops'] = df['Total_Stops'].map({'non-stop': 0, '1 stop': 1, '2 stops': 2, '3 stops': 3,'4 stops':4})
df['Total_Stops'] = df['Total_Stops'].astype('int')

In [36]:
# Merging 'No Info'into 'No info'
df['Additional_Info'] = df['Additional_Info'].replace('No Info', 'No info')

In [37]:
#Merging 'Delhi' and 'New Delhi'
df['Destination'] = df['Destination'].replace('Delhi', 'New Delhi')

In [38]:
df['Additional_Info'].value_counts()

No info                         8347
In-flight meal not included     1982
No check-in baggage included     320
1 Long layover                    19
Change airports                    7
Business class                     4
Red-eye flight                     1
1 Short layover                    1
2 Long layover                     1
Name: Additional_Info, dtype: int64

In [39]:
# Dropping a row which have 'Duration == 5m'
df.drop(df.loc[df['Duration'] == '5m'].index, inplace=True)

In [40]:
#Creating a function which can convert 'Duration' into minutues `eg: 2h 50m to 170 minutes`
from collections import defaultdict
import re

def humantime2minutes(s):
    d = {
      'w':      7*24*60,
      'week':   7*24*60,
      'weeks':  7*24*60,
      'd':      24*60,
      'day':    24*60,
      'days':   24*60,
      'h':      60,
      'hr':     60,
      'hour':   60,
      'hours':  60,
    }
    mult_items = defaultdict(lambda: 1).copy()
    mult_items.update(d)

    parts = re.search(r'^(\d+)([^\d]*)', s.lower().replace(' ', ''))
    if parts:
        return int(parts.group(1)) * mult_items[parts.group(2)] + humantime2minutes(re.sub(r'^(\d+)([^\d]*)', '', s.lower()))
    else:
        return 0

In [41]:
# We can also use the below function to which can convert 'Duration' into minutues
"""
def duration(test):
    test = test.strip()
    total=test.split(' ')
    to=total[0]
    hrs=(int)(to[:-1])*60
    if((len(total))==2):
        mint=(int)(total[1][:-1])
        hrs=hrs+mint
    test=str(hrs)
    return test
train['Duration']=train['Duration'].apply(duration)

"""

"\ndef duration(test):\n    test = test.strip()\n    total=test.split(' ')\n    to=total[0]\n    hrs=(int)(to[:-1])*60\n    if((len(total))==2):\n        mint=(int)(total[1][:-1])\n        hrs=hrs+mint\n    test=str(hrs)\n    return test\ntrain['Duration']=train['Duration'].apply(duration)\n\n"

In [42]:
# Applying the above function to 'Duration' column
df['Duration'] = df['Duration'].apply(humantime2minutes)
df['Duration'] = df['Duration'].astype(int)

In [43]:
corr = df.corr()
a1 = sns.heatmap(corr, cmap='hot') 
a1.set_title('Correlation Matrix')

Text(0.5, 1.0, 'Correlation Matrix')

In [44]:
ax = sns.countplot(df['Month_of_Journey']) ## This Plot concludes that '5 or (May)' month have maximum jouney happend

for p in ax.patches:
    ax.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

C:\Users\Sudheesh E\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


In [45]:
a2 = sns.barplot(x ='Month_of_Journey', y = 'Price',data = df, estimator=sum)
a2.set_title('Month vs Price')
a2.set_ylabel('Price')
a2.set_xlabel('Month of booking')   


for p in a2.patches:
    a2.annotate(int(p.get_height()), (p.get_x()+0.25, p.get_height()+1), va='bottom',
                    color= 'black')

# This plot shows the if we take Sum of all the price in each month May month have Highest Total
# Lowest is on April

In [46]:
#Month_of_Journey v/s Averagefare

monthly_avg=df.groupby(['Month_of_Journey']).agg({'Price':np.mean}).reset_index()

a3 = monthly_avg.plot(x='Month_of_Journey',y='Price',figsize=(6,6))
a3.set_title('Month_of_Journey v/s Averagefare');

## Average price is highest in the 'March' month and Lowest is on 'April'

In [47]:
sns.catplot(x = 'Destination', y = 'Price', data = df.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=15/6)

## New Delhi have highest Price

In [48]:
sns.catplot(x = 'Source', y = 'Price', data = df.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=15/6)

In [49]:
sns.catplot(x = 'Airline', y = 'Price', data = df.sort_values('Price',ascending=False),kind="boxen",height=6, aspect=15/6)
# Jet Airways Business has the Highest Price

In [50]:
sns.boxplot(df['Duration']) ## Few otliers are present

C:\Users\Sudheesh E\anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


<AxesSubplot:xlabel='Duration', ylabel='Price'>

In [51]:
sns.histplot(df['Duration'], kde = True)

<AxesSubplot:xlabel='Duration', ylabel='Price'>

In [52]:
sns.relplot(x = df['Duration'], y = df['Price'])

In [53]:
df.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,IndiGo,Banglore,New Delhi,night,night,170,0,No info,3897,3,24,6
1,Air India,Kolkata,Banglore,morning,afternoon,445,2,No info,7662,5,1,2
2,Jet Airways,Delhi,Cochin,morning,night,1140,2,No info,13882,6,9,6
3,IndiGo,Kolkata,Banglore,evening,night,325,1,No info,6218,5,12,6
4,IndiGo,Banglore,New Delhi,evening,night,285,1,No info,13302,3,1,4


In [54]:
ax = sns.barplot(x = 'Arrival_Time', y = 'Price', data = df)
ax.set_title('Arrival_Time vs Price')

Text(0.5, 1.0, 'Arrival_Time vs Price')

In [55]:
ax = sns.barplot(x = 'Dep_Time', y = 'Price', data = df)
ax.set_title('Departure_Time vs Price')

Text(0.5, 1.0, 'Departure_Time vs Price')

In [56]:
ax = sns.barplot(x = 'Total_Stops', y = 'Price', data = df)
ax.set_title('Total_Stops vs Price')

Text(0.5, 1.0, 'Total_Stops vs Price')

In [57]:
ax = sns.barplot(x = 'Weekday_of_Journey', y = 'Price', data = df)  # Price on '4' and '6' is high that is on (Fridays and Sundays)
ax.set_title('Weekday_of_Journey vs Price')

Text(0.5, 1.0, 'Weekday_of_Journey vs Price')

### Feature Engineering

In [58]:
label_encoder = LabelEncoder()

In [59]:
df['Airline'] = label_encoder.fit_transform(df['Airline'])
df['Source'] = label_encoder.fit_transform(df['Source'])
df['Destination'] = label_encoder.fit_transform(df['Destination'])
df['Additional_Info'] = label_encoder.fit_transform(df['Additional_Info'])
df['Dep_Time'] = label_encoder.fit_transform(df['Dep_Time'])
df['Arrival_Time'] = label_encoder.fit_transform(df['Arrival_Time'])

In [60]:
# Dropping records which have less value_counts based on 'Airline' column

counts = df['Airline'].value_counts()

df_new = df[~df['Airline'].isin(counts[counts < 15].index)]

df_new.reset_index()

# This is another method to do the filtering of rows based on valuecounts
# df.groupby('Airline').filter(lambda x : len(x)<15)

,index,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,0,3,0,4,3,3,170,0,7,3897,3,24,6
1,1,1,3,0,2,0,445,2,7,7662,5,1,2
2,2,4,2,1,2,3,1140,2,7,13882,6,9,6
3,3,3,3,0,1,3,325,1,7,6218,5,12,6
4,4,3,0,4,1,3,285,1,7,13302,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10653,10678,0,3,0,1,3,150,0,7,4107,4,9,1
10654,10679,1,3,0,1,3,155,0,7,4145,4,27,5
10655,10680,4,0,4,2,0,180,0,7,7229,4,27,5
10656,10681,10,0,4,0,0,160,0,7,12648,3,1,4


In [61]:
# Dropping records which have less value_counts based on 'Additional_Info' column

counts = df_new['Additional_Info'].value_counts()

df_new = df_new[~df_new['Additional_Info'].isin(counts[counts < 20].index)]

df_new.reset_index()

,index,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,0,3,0,4,3,3,170,0,7,3897,3,24,6
1,1,1,3,0,2,0,445,2,7,7662,5,1,2
2,2,4,2,1,2,3,1140,2,7,13882,6,9,6
3,3,3,3,0,1,3,325,1,7,6218,5,12,6
4,4,3,0,4,1,3,285,1,7,13302,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10623,10678,0,3,0,1,3,150,0,7,4107,4,9,1
10624,10679,1,3,0,1,3,155,0,7,4145,4,27,5
10625,10680,4,0,4,2,0,180,0,7,7229,4,27,5
10626,10681,10,0,4,0,0,160,0,7,12648,3,1,4


In [62]:
# Dropping records which have less value_counts based on 'Additional_Info' column

counts = df_new['Total_Stops'].value_counts()

df_new = df_new[~df_new['Total_Stops'].isin(counts[counts < 50].index)]

df_new.reset_index()

,index,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,0,3,0,4,3,3,170,0,7,3897,3,24,6
1,1,1,3,0,2,0,445,2,7,7662,5,1,2
2,2,4,2,1,2,3,1140,2,7,13882,6,9,6
3,3,3,3,0,1,3,325,1,7,6218,5,12,6
4,4,3,0,4,1,3,285,1,7,13302,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10578,10678,0,3,0,1,3,150,0,7,4107,4,9,1
10579,10679,1,3,0,1,3,155,0,7,4145,4,27,5
10580,10680,4,0,4,2,0,180,0,7,7229,4,27,5
10581,10681,10,0,4,0,0,160,0,7,12648,3,1,4


In [63]:
df_new.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,3,0,4,3,3,170,0,7,3897,3,24,6
1,1,3,0,2,0,445,2,7,7662,5,1,2
2,4,2,1,2,3,1140,2,7,13882,6,9,6
3,3,3,0,1,3,325,1,7,6218,5,12,6
4,3,0,4,1,3,285,1,7,13302,3,1,4


In [64]:
# pandas_profiling.ProfileReport(df_new)

# import dtale
# d = dtale.show(df_new)
# d.open_browser()

# import sweetviz
# my_report  = sweetviz.analyze(df_new, target_feat='Price')
# my_report.show_html('FinalReport.html')

In [65]:
from sklearn.feature_selection import mutual_info_classif

x = df_new.drop('Price', axis= 'columns')
y = df_new['Price']


In [66]:
# Looking for Feature importance based on 'mutual_info_classif' from Sklearn

imp = pd.DataFrame(mutual_info_classif(x,y), index= x.columns)

imp.sort_values(by=0, ascending= False)

,0
Airline,2.550874
Destination,2.437155
Source,2.383473
Duration,2.371678
Total_Stops,2.150526
Additional_Info,1.501113
Month_of_Journey,1.248450
Dep_Time,1.120641
Arrival_Time,1.045301
Day_of_Journey,0.787552


In [70]:
# df_new.to_csv('final_data.csv', index= False)

In [73]:
data = pd.read_csv('final_data.csv')

In [74]:
data.head()

,Airline,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Month_of_Journey,Day_of_Journey,Weekday_of_Journey
0,3,0,4,3,3,170,0,7,3897,3,24,6
1,1,3,0,2,0,445,2,7,7662,5,1,2
2,4,2,1,2,3,1140,2,7,13882,6,9,6
3,3,3,0,1,3,325,1,7,6218,5,12,6
4,3,0,4,1,3,285,1,7,13302,3,1,4
